# 0. はじめに
このノートは「Open Problems - Multimodal Single-Cell Integration」の課題を理解するために、母国語である日本語で内容を整理する目的で作成しています。専門用語が多いので調べながら進めていきます。理解が間違っていることに気づいたりしたら、コメントで是非教えてください。ちょっとずつ更新しています。最終更新日は9月27日。
<br>
<br>
<br>


## 課題要約
* 細胞ではDNA→RNA→たんぱく質の順に情報が翻訳されて物質が生成されます
* `train_multi_inputs.h5`というファイルを使ってDNAに関連する量からRNAに関連する量を予測するモデルを作ります
* `train_cite_inputs.h5`というファイルを使ってRNAに関連する量からたんぱく質に関連する量を予測するモデルを作ります
* 課題の意義としては、測定量を使って幹細胞が何の役割を持った細胞に分化するかしれたらいいなぁみたいな気持ち
<br>
<br>
<br>

# 1. OverView

## 1.1 Description
### Goal of the Competition（コンペの目標）
このコンペの目的は、**骨髄幹細胞がより成熟した血液細胞へと成長する過程で、DNA、RNA、タンパク質の測定値が単一細胞内でどのように変化するかを予測する**ことです。競技参加者が行うことは、4人のヒトドナーから5つの時点で採取したCD34+造血幹・前駆細胞（HSPC）の30万細胞の時系列データセットで学習したモデルを開発することになります。このデータは創薬企業Cellarity社がこのコンペのために作成したものです。

> **_NOTE:_**  骨髄幹細胞は造血幹細胞のことを指しているようです。造血幹細胞は分化すると最終的に赤血球や白血球、血小板などの血液関連の細胞になります。

<br>
<br>


テスト用のデータセットはデータセット内の未知の後期時点から採取されています。競技参加者は1つの様式(modality)を与えられ、同じ細胞で測定された対になる様式を予測することが課されることになります。このコンペティションの挑戦は、テストデータがトレーニングデータのどの時点よりも遅い時点のものであることです。

> **_NOTE:_**  様式(modality)はゲノムモダリティのことでしょうか。ゲノムモダリティとは、ゲノムの構造や機能の事のようです。図を見ると、測定量から遺伝子発現量やたんぱく質量を予測するという課題のようです。細胞の学習していない細胞後期の状態を予測するところがチャレンジングな点です。

<br>
<br>

この解析により、細胞の状態を何層にも分けて遺伝情報をマッピングする手法の革新を加速させると見込まれます。もしある様式を別の様式から予測することができれば、これらの複雑な制御プロセスを支配する規則についての理解を深めることができるかもしれません。


<br>
<br>
<br>

### Context(背景)
この過去10年において、シングルセルゲノミクスの登場により単一細胞内のDNA、RNA、タンパク質の測定が可能になりました。そしてこれらの技術によってこれまでにないスケールと解像度で生物学を研究出来るなりました。その結果、ヒトの初期胚発生の詳細なマップ、新しい疾患関連細胞の発見、細胞を標的とした治療介入などが可能となりました。さらに、近年の実験技術の進歩により、同一細胞内で複数のゲノムモダリティを測定することが可能になりました。


このようにマルチモーダルな単一細胞データが利用できるようになった一方で、データ解析の手法はまだ乏しいのが現状です。単一細胞の体積は小さいために測定値はまばらでノイズが多く、細胞間の分子サンプリング深さの違い（シーケンシング深さ）やバッチで細胞を扱うことによる技術的影響（バッチ効果）は生物学的な差異を分からなくしてしまう事が多々あります。マルチモーダルデータを解析する場合、異なる特徴空間やモダリティ間およびバッチ間における共有および固有の変動を考慮する必要があります。さらに、現在の単一細胞データ解析経路はたとえその下にダイナミックな生物学的プロセスがあったとしても細胞を静的なスナップショットとして扱っています。時間の経過に伴う状態の変化と同時に時間的なダイナミクスを考慮することは、単一細胞のデータサイエンスにおける未解決の課題です。


一般に、遺伝情報はDNA→RNA→たんぱく質と流れます。DNAはRNAを生成するために読み取り可能でなければならず（ATACデータ）、RNAはたんぱく質を生成するためにテンプレートとして使用されます（ADTデータ）。これらのプロセスはフィードバックによって制御されています。例えば、たんぱく質がDNAと結合することでより多くのRNAが生成されるのを防ぐことができます。このような遺伝子の制御が、生物が発展し環境変化に適応していくための動的な細胞プロセスの基盤となっています。単一細胞データ科学では、動的プロセスは生物学的プロセスの進行を捕らえるためのいわゆる擬似時間アルゴリズムによってモデル化されてきました。しかし、これらのアルゴリズムを一般化し擬似時間と実時間の両方を考慮することはまだ未解決の問題です。


コンペティション・ホストのOpen Problems in Single-Cell Analysisは、単一細胞解析手法のベンチマークを標準化するためのオープンソースでコミュニティ主導の取り組みです。Open Problemsの中核となる取り組みには、既存の課題を測定可能なタスクに形式化すること、高品質のデータセットを収集すること、コミュニティから提供された手法のベンチマークを集中的に行うこと、多様な手法開発者を集めて単一細胞のアルゴリズムを改善するコミュニティに焦点を当てたイベントを開催することなどがあります。Cellarity、Chan Zuckerbeg Biohub、Chan Zuckerberg Initiative、Helmholtz Munich、Yaleと提携し、学際的なコラボレーションを通じて遺伝的動態の時間的変化の予測にどんな進展が見られるか期待してます。

人体には約37兆個の細胞が存在しそれぞれが異なる行動や機能をもっています。一つのゲノムがどのようにして多様な細胞状態を生み出すのかを理解することは、健康や病気において組織がどのように機能し、どのように誤作動を起こすのか、メカニズムに対する洞察を得るための鍵となります。競技参加者はこのコンペに参加することによりこの単一細胞生物学の基本的な課題の解決に貢献することができます。予測問題を長期にわたって解くことができれば、血液や免疫細胞の成熟に伴う分化に遺伝子制御がどのように影響するかについて新たな知見を得ることができるかもしれません。

コンペティションのヘッダー画像：Pawel Czerwinski on Unsplash


> **_NOTE:_**  単一細胞測定が発達したものの、測定ノイズが各細胞間の違いよりも大きいことがあり解析を難しくしています。また、細胞は常に変化しているため、ある時刻の状態よりもダイナミクスのほうがより情報があり、個体スケールで現れる適応の基盤にもなっています。細胞ダイナミクスを実時間でもモデル化することで理解できることが多くなるようです（具体的にはよくわからないですが）。

<br>
<br>
<br>

## 1.2 Evaluation（評価）
提出した成果物の順位付けには、ピアソン相関係数を使用します。Multiomeデータセットの各観察について、正解の分かった遺伝子発現と予測される遺伝子発現の相関を計算します。CITEseqデータセットの各観察について、正解の分かった表面たんぱく質レベルと予測される表面たんぱく質レベルの間の相関を計算します。総合点数は各サンプルの相関スコアの平均値です。あるサンプルの予測値がすべて同じであった場合、そのサンプルの相関は-1.0として点数化されます。

> **_NOTE:_**  ピアソン相関係数$$\rho=\frac{\sum_i (x_i-{\bar x})(y_i-{\bar y})}{\sqrt{\sum_i(x_i-{\bar x})^2 \sum_j(y_j-{\bar y})^2 }}$$



### Submission File（提出ファイル）
評価セット内の各`id`について対応する`row_id`の`target`を予測する必要があります。提出物はヘッダーを含み、以下のフォーマットである必要があります。

```
row_id,target
0,0.0
1,0.0
2,0.0
3,0.0
...
```

提出ファイルはテストセットの観察のサブセットのみを含む必要があります。含まれるべき特定のID については、データ説明を参照してください。

## 1.3 Timeline
日本時間  
開始日　　　　　　　：2022年 8月16日  
エントリー締め切り　：2022年11月9日 8:59  
チームマージ締め切り：2022年11月9日 8:59  
提出締め切り　　　　：2022年11月16日 8:59

## 1.4 Prizes
このコンペティションは、NeurIPS 2022 コンペティショントラックの一部です。優秀なメソッドの賞金は次のとおりです。

* 1位 - 15,000ドル
* 2位 - 7,000ドル
* 3位 - 3,000ドル

賞金に加えて、受賞者は 11 月にバーチャルで開催される NeurIPS 2022 コンペティション ワークショップに招待されます。

# 2. Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import scipy
import time

! pip install -q tables  # needed for loading HDF files

## Data Description
このコンペのデータセットは、4人の健康なヒトドナーから分離した動員末梢CD34+造血幹細胞および前駆細胞（HSPCs）から収集した単一細胞マルチオミクスデータから構成されています。細胞についての詳細は、ベンダーのウェブサイトに掲載されています。


測定は10日間にわたり5つの時点で行われ、各時点の各プレートから2つの単一細胞アッセイ手法で測定するために細胞を採取ました。1つ目は10x Chromium Single Cell Multiome ATAC + Gene Expression technology（Multiome）、2つ目は10x Genomics Single Cell Gene Expression with Feature Barcoding technologyで、TotalSeq™-B Human Universal Cocktail, V1.0 (CITEseq) を使用したものです。


このデータ型を扱ったことがない方のために、この説明の下にいくつかのリンクを掲載しました。

各アッセイ技術では、2つのモダリティを測定します。**Multiomeキットはクロマチンアクセシビリティ（DNA）と遺伝子発現（RNA）を測定し、CITEseqキットは遺伝子発現（RNA）と表面タンパク質のレベルを測定**します。

> **_NOTE:_** 私の理解では、クロマチンアクセシビリティとはＤＮＡの読み取り可能性を表す。クロマチンはDNAとたんぱく質で構成されており、たんぱく質が結合することによってDNAは折りたたまれ読み取りが制御されることで不必要な配列を転写しないようにしている。

> **_NOTE:_** 遺伝子発現（RNA）はRNAの量の指標を指していると思われる。RNAはDNAからの転写で生成されるのでクロマチンアクセシビリティと関係があると推測される。

> **_NOTE:_** 表面たんぱく質レベルはおそらく膜タンパク質の量の指標。どのたんぱく質を測定しているのかなどはまだ理解していない。たんぱく質はRNAから合成されるので、遺伝子発現と表面たんぱく質は関係があると考えられる。

分子生物学のセントラルドグマに従うと DNA→RNA→タンパク質という分子生物学のセントラルドグマに従って、以下のようなタスクがあります。

* Multiomeサンプルについては、DNAからRNAを予測
* CITEseqサンプルについては、RNAからたんぱく質量を予測

<br>
<br>
<br>

### Cell Types 細胞の種類
解析の参考のため、以下の論文の情報を用いて、RNA 遺伝子発現に基づく予備的な細胞タイプのアノテーションを行った（https://www.nature.com/articles/ncb3493）。
細胞タイプのアノテーションは不正確な技術であり、連続データに離散的なラベルを割り当てることは限界があることに注意してください。これらのラベルを予測に使用する必要はなく、主に探索的な解析のガイドとして提供されています。このデータでは、以下のような細胞タイプがある。

* HSC = Hematoploetic Stem Cell（造血幹細胞）
* EryP = Erythrocyte Progenitor（赤血球前駆体）
* NeuP = Neutrophil Progenitor（好中球前駆体）
* MasP = Mast Cell Progenitor（マスト細胞前駆体）
* MkP = Megakaryocyte Progenitor（巨核球前駆体）
* BP = B-Cell Progenitor（B細胞前駆体）
* MoP = Monocyte Progenitor（単球前駆体）
* hidden

> **_NOTE:_** ○○前駆体（前駆細胞）は造血幹細胞から○○細胞に分化する途中の状態の細胞です。細胞の時間変化の不連続な状態に名前を付けているだけなので予測に使う必要はないとされています。しかし、一時的な分類であっても問題を解くにあたって情報を整理することは役に立つかもしれないので、造血幹細胞の分化の略式系統樹を記しておきます。  
<span style="color: blue; ">HSC</span>
――> 骨髄系前駆細胞 -> <span style="color: blue; ">EryP</span>  
　　　|　　　　　　　　　-> <span style="color: blue; ">MasP</span>  
　　　|　　　　　　　　　-> <span style="color: blue; ">MkP</span>  
　　　|　　　　　　　　　-> 骨髄芽球 ――> <span style="color: blue; ">NeuP</span>  
　　　|　　　　　　　　　　　　　　　――> <span style="color: blue; ">MoP</span>  
 　　　―> リンパ系前駆細胞 -> <span style="color: blue; ">BP</span> 
    
<br>
<br>
<br>


## File and Field Descriptions（ファイルおよびフィールドの説明）
**metadata.csv**
* `cell_id` - 観測された各細胞の識別子。
* `donor` - 4つの細胞ドナーの識別子。
* `day` - 観察が行われた実験の日。
* `technology` - citeseqまたはmultiomeのどちらか。
* `cell_type` - 上記細胞タイプの1つ、または非表示。


In [ ]:
metadata = pd.read_csv('../input/open-problems-multimodal/metadata.csv').set_index('cell_id')
print('length of metadata :',len(metadata))
print('# of unique cell_id:',len(metadata.index.unique()))
metadata.head()

<br>
<br>
<br>


実験結果はいくつかの大きなアレイに格納されており、HDF5フォーマットで提供されています。


### Multiome
* **train/test_multi_inputs.h5**  
ATAC-seqピーク数をデフォルトのlog(TF) * log(IDF) 出力（DNA）を用いてTF-IDF変換したもの。行は細胞、列はアクセスレベル(DNA)を測定したゲノム上の位置（ここでは10x References - 2020-A (July 7, 2020) で提供した参照ゲノムGRCh38のゲノム座標で特定）に相当する。

> **_NOTE:_** TF-IDF変換：統計量への変換。この文脈ではDNA配列において特定の配列がいかに重要なのかを示す指標への変換。（ただしDNA配列の区切りがどこにあるのか理解していません）TFは特定配列の出現頻度、IDFは各区切りで特定配列が出現する割合で$log(TF) * log(IDF)$で表す。

> **_NOTE:_** ゲノム座標：ゲノム座標は参照ゲノムの染色体名や開始位置、および終了位置が次の形式で含まれたもの。`chr1:1234570-1234870`



In [ ]:
df_multi_in = pd.read_hdf('../input/open-problems-multimodal/train_multi_inputs.h5', start=0,stop=30).astype(np.float16)
df_multi_in.head()


* **train_multi_targets.h5**  
RNA遺伝子発現レベル（同一細胞のライブラリサイズ正規化およびlog1p変換したカウント）。

> **_NOTE:_** ライブラリサイズ正規化：同一細胞内の総和が１になるような正規化。

> **_NOTE:_** log1p変換：$f(x) = log(x+1)$

In [ ]:
df_multi_tg = pd.read_hdf('../input/open-problems-multimodal/train_multi_targets.h5', start=0,stop=30)
df_multi_tg.head()

<br>
<br>
<br>


### CITEseq
* **train/test_cite_inputs.h5**  
RNAのライブラリサイズ正規化およびlog1p変換されたカウント（遺伝子発現量）で、行は細胞、列は`{gene_name}_{gene_ensemble-ids}`で与えられた遺伝子に対応する。

> **_NOTE:_** gene_ensemble-idsがよくわからない。gene_ensembleとは何か、なぜgene_ensembleで区別しているのか。


In [ ]:
df = pd.read_hdf('../input/open-problems-multimodal/train_cite_inputs.h5', start=0,stop=30)
df.head()


* **train_cite_targets.h5**  
dsbで正規化された同じ細胞の表面タンパク質レベル。

> **_NOTE:_** dsb：たんぱく質信号の背景処理

In [ ]:
df = pd.read_hdf('../input/open-problems-multimodal/train_cite_targets.h5', start=0,stop=30)
df.head()

<br>
<br>
<br>

## Splits　（データ分割）
データの分割は以下のように行われる。

* トレーニングセットはIDが13176, 31800, 32606のドナーのみから構成されます。公開テストセットは、ドナーIDが27678からのサンプルだけからなります。プライベートテストセットは、4人全てのドナーからのサンプルで構成されています。

* Multiomeサンプルについては、トレーニングセットは2日目、3日目、4日目、7日目のサンプルのみから構成されています。公開テストセットは2日目、3日目、7日目のみのサンプルから構成されています。プライベートテストセットは、10日目のデータのみから構成されています。

* CITEseqサンプルについては、トレーニングセットは2日目、3日目、4日目のサンプルのみから構成されています。公開テストセットも2日目、3日目、4日目のみのサンプルで構成される。プライベートテストセットは、7日目のサンプルだけから構成されています。どの分割においても、10日目のCITEseqサンプルは存在しません。

競技参加者の課題は、**テストセットの入力に対応するラベルを予測する**ことです。提出物の採点を容易にするため、Multiomeのサブセットに対する予測のみを要求します。このサブセットは、Multiomeの行の30%、各行に対して列の15%をサンプリングして作成されています。列のサンプルは行ごとに異なります。CITEseqラベルは全てスコア化されている。

* **evaluation_ids.csv** - 評価対象のテストセットからラベルを特定します。ラベルマトリックスのcell_id / gene_id 識別子から、提出ファイルに必要なrow_idへの結合キーを提供します。
* **sample_submission.csv** - 正しいフォーマットのサンプル提出ファイル。詳細は、「評価」のページを参照してください。

In [ ]:
evaluation_ids = pd.read_csv('../input/open-problems-multimodal/evaluation_ids.csv').set_index('row_id')
evaluation_ids.head()

# EDA
metadataの中身を見てみる。細胞の種類はどのようになっている？

In [ ]:
fig, ax = plt.subplots()
sns.countplot(data=metadata, x ='cell_type', ax=ax)
plt.show()

<span style="color: blue; ">HSC</span>
――> 骨髄系前駆細胞 -> <span style="color: orange; ">EryP</span>  
　　　|　　　　　　　　　-> <span style="color: red; ">MasP</span>  
　　　|　　　　　　　　　-> <span style="color: purple; ">MkP</span>  
　　　|　　　　　　　　　-> 骨髄芽球 ――> <span style="color: green; ">NeuP</span>  
　　　|　　　　　　　　　　　　　　　――> <span style="color: pink; ">MoP</span>  
 　　　―> リンパ系前駆細胞 -> <span style="color: brown; ">BP</span> 

分化前のHSCが最も多い。細胞分化には時間方向があるので、時間変化を見てみる。まず、各ドナーの細胞数の時間変化。

In [ ]:
def time_ev_of_num_cells():
    mlist=['x','o','s','^']
    clist=['royalblue','orangered','hotpink','limegreen']

    fig, ax = plt.subplots()
    donors = metadata['donor'].unique()
    for i in range(len(donors)):
        df_donor=metadata[metadata['donor']==donors[i]]
        dnr = 'donor#'+str(donors[i])

        group_by_day = df_donor.groupby('day').sum()

        ax.plot(group_by_day.index, group_by_day['donor'],
                marker=mlist[i%len(mlist)],ms=13,lw=0.5,
                color=clist[i%len(clist)],markerfacecolor='None',
                label=dnr
               )
    ax.set_xlabel('day',fontsize=12)
    ax.set_ylabel('counts',fontsize=12)
    title = 'time ev. of #cells of each donor'
    ax.set_title(title)
    ax.legend(loc=(1.01,0.8))
    
time_ev_of_num_cells()
plt.show()

ドナーによって2倍程度の細胞数の差がある。４人中３人で、4日目でピークが来ている。


次に各ドナー各細胞種類ごとの時間変化。

In [ ]:
def time_ev_of_celltype(ctype):
    mlist=['x','o','s','^']
    clist=['royalblue','orangered','hotpink','limegreen']

    fig, ax = plt.subplots()
    donors = metadata['donor'].unique()
    for i in range(len(donors)):
        df_donor=metadata[metadata['donor']==donors[i]]
        dnr = 'donor#'+str(donors[i])

        df_donor_hsc = df_donor[df_donor['cell_type']==ctype]
        group_by_day = df_donor_hsc.groupby('day').sum()

        ax.plot(group_by_day.index, group_by_day['donor'],
                marker=mlist[i%len(mlist)],ms=13,lw=0.5,
                color=clist[i%len(clist)],markerfacecolor='None',
                label=dnr
               )
    ax.set_xlabel('day',fontsize=12)
    ax.set_ylabel('counts',fontsize=12)
    title = 'time ev. of # of '+ctype
    ax.set_title(title)
    ax.legend(loc=(1.01,0.8))
    
ctypes = metadata['cell_type'].unique()

for ctype in ctypes:
    time_ev_of_celltype(ctype)

* 減少傾向にあるもの：HSC,BP
* 増加傾向にあるもの：MasP,MoP

<br>
<br>
<br>


# データに触る
## Multiomeのファイルが大きい
Multiomeのtrainファイルが大きすぎてメモリ不足になるため、工夫をしなければならない。[上記](###Multiome)で見たように、`df_multi_in`は0のセルが多く疎行列(sparse matrix)である。0以外のセルを扱うことでメモリを節約できる。CSR方式という圧縮方法を試してみる。以下、`while True`にして全部読み込むのにはだいたい30分ほどで処理できる。時間がかかるから`for i in range(3)`で３つだけ読み込んでいる。

In [ ]:

time_start = time.time()
fname_multi = '../input/open-problems-multimodal/train_multi_inputs.h5'
chunksize = 5000


# initial state
csr = scipy.sparse.csr_matrix([])
n_start = 0
n_end   = n_start+chunksize
#while True:
for i in range(3):
    # reading part of files
    df_chunk  = pd.read_hdf(fname_multi, start=n_start,stop=n_end).astype(np.float16)
    # converting from dataframe to csr
    csr_chunk = scipy.sparse.csr_matrix(df_chunk.to_numpy())
          
    # stacking data
    if (csr).shape[1]==0:
        csr = csr_chunk
    else:
        csr = scipy.sparse.vstack([csr, csr_chunk])
        
    # updating state
    n_start = n_end
    n_end   = n_start+chunksize
    
    # calculation time of process
    time_now = time.time()
    time_process = time_now -time_start
    print(n_start,' time:', round(time_process/60, 2),'min')
    
    # check the end of the file
    if len(df_chunk)<chunksize:
        print('complete')
        print(csr.shape)
        del df_chunk
        break
#savename = '/tmp/train_multi_inputs.npz'
#scipy.sparse.save_npz(savename, csr)

In [ ]:
savename = '/tmp/train_multi_inputs.npz'
scipy.sparse.save_npz(savename, csr)

In [ ]:
time_start = time.time()
savename = '/tmp/train_multi_inputs.npz'
test = scipy.sparse.load_npz(savename)
# calculation time of process
time_now = time.time()
time_process = time_now -time_start
print(' time:', round(time_process/60, 2),'min')

In [ ]:
print(test)

上記の方法で疎行列を作成することができるが、kaggleでは他の人が加工したデータを利用することができる。例えば、自分で作成したNotebookの右上の矢印をクリックすると、Dataという項目があり(+Add Data)というボタンをクリックすると他の参加者が公開しているデータをNotebook内で利用できるようになる。本コンペティションでは例えば「Multimodal Single Cell as Sparse Matrices」というFABIEN CROM氏が作成したデータセットなどが非常に使いやすい。試しにこのデータセットのCITEのtrainデータを読み込んでみる。

In [ ]:
# File names
path   = "../input/multimodal-single-cell-as-sparse-matrix/"
f_trci = path+'train_cite_inputs_values.sparse.npz'
f_trct = path+'train_cite_targets_values.sparse.npz'
f_teci = path+'test_cite_inputs_values.sparse.npz'

# loading train data
trci = scipy.sparse.load_npz(f_trci)
trct = scipy.sparse.load_npz(f_trct)

なんと簡単。